In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime


## Create Dataframe and define Schema

In [8]:
InputData = [
    (1,'Prasad Nadig', 25, 'NJ','2022-01-01', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")),
    (2,'Ethereum', 80, 'NY', '2022-01-02', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")),
    (3,'Cosmos', 25, 'PA', '2022-01-03', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")),
    (4,'Solana', 55, 'MD', '2022-01-04', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")),
    (5,'Carnado', 15, 'TX', '2022-01-05', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")),
    (6,'Link', 45, 'NJ', '2022-01-06', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S"))
]

#Define schema for the source data
schema = StructType([ \
    StructField("cust_id",IntegerType(),True), \
    StructField("cust_name",StringType(),True), \
    StructField("cust_age",IntegerType(),True), \
    StructField("cust_loc",StringType(),True), \
    StructField("create_date", StringType(), True), \
    StructField("last_updated_time", TimestampType(), True)
  ])

#Create dataframe from the input data and the corresponding schema
inputDF = spark.createDataFrame(data=InputData,schema=schema)

In [9]:
#Check data 
inputDF.show()

+-------+------------+--------+--------+-----------+-------------------+
|cust_id|   cust_name|cust_age|cust_loc|create_date|  last_updated_time|
+-------+------------+--------+--------+-----------+-------------------+
|      1|Prasad Nadig|      25|      NJ| 2022-01-01|2022-11-28 15:06:58|
|      2|    Ethereum|      80|      NY| 2022-01-02|2022-11-28 15:06:58|
|      3|      Cosmos|      25|      PA| 2022-01-03|2022-11-28 15:06:58|
|      4|      Solana|      55|      MD| 2022-01-04|2022-11-28 15:06:58|
|      5|     Carnado|      15|      TX| 2022-01-05|2022-11-28 15:06:58|
|      6|        Link|      45|      NJ| 2022-01-06|2022-11-28 15:06:58|
+-------+------------+--------+--------+-----------+-------------------+



## Define HUDI options, write data to S3 as HUDI dataset - INSERT

In [10]:
hudiOptions = {
'hoodie.table.name': 'customer',
'hoodie.datasource.write.recordkey.field': 'cust_id',
'hoodie.datasource.write.partitionpath.field': 'create_date',
'hoodie.datasource.write.precombine.field': 'last_updated_time',
'hoodie.datasource.hive_sync.enable': 'true',
'hoodie.datasource.hive_sync.use_jdbc': 'false',
'hoodie.datasource.hive_sync.mode':'hms',
'hoodie.datasource.hive_sync.table': 'customer',
'hoodie.datasource.hive_sync.partition_fields': 'create_date',
'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}

In [11]:
inputDF.write \
.format('org.apache.hudi') \
.option('hoodie.datasource.write.operation', 'insert') \
.options(**hudiOptions) \
.mode('overwrite') \
.save('s3://emr-studio-emr-on-eks/hudi-tables/')

In [ ]:
#datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")

## Read data from HUDI Dataset we just created

In [12]:
# By default HUDI performs snapshot queries. 
snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*')
    
snapshotQueryDF.select("cust_id", "cust_name", "cust_age", "cust_loc", "create_date", "last_updated_time").orderBy("cust_id").show()

+-------+------------+--------+--------+-----------+-------------------+
|cust_id|   cust_name|cust_age|cust_loc|create_date|  last_updated_time|
+-------+------------+--------+--------+-----------+-------------------+
|      1|Prasad Nadig|      25|      NJ| 2022-01-01|2022-11-28 15:06:58|
|      2|    Ethereum|      80|      NY| 2022-01-02|2022-11-28 15:06:58|
|      3|      Cosmos|      25|      PA| 2022-01-03|2022-11-28 15:06:58|
|      4|      Solana|      55|      MD| 2022-01-04|2022-11-28 15:06:58|
|      5|     Carnado|      15|      TX| 2022-01-05|2022-11-28 15:06:58|
|      6|        Link|      45|      NJ| 2022-01-06|2022-11-28 15:06:58|
+-------+------------+--------+--------+-----------+-------------------+



# DML Operations

## UPSERT
###  - HUDI write opearation provides 3 options Upsert/Insert and Bulk Insert, we did Insert in the previous steps, now lets try the upsert operation

In [13]:
# We will update an existing record and insert a new record. Upsert operation in HUDI will find the record based on the RecordKey, if found it will update the value, if not found then will Insert the record.
InputData = [
    (1,'Prasad S Nadig', 30, 'NJ','2022-01-01', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")), #Update
    (7,'Compound', 20, 'NJ', '2022-01-07', datetime.strptime(datetime.now().strftime("%Y-%d-%m %H:%M:%S"), "%Y-%d-%m %H:%M:%S")) #Insert
]

#Define schema for the source data
schema = StructType([ \
    StructField("cust_id",IntegerType(),True), \
    StructField("cust_name",StringType(),True), \
    StructField("cust_age",IntegerType(),True), \
    StructField("cust_loc",StringType(),True), \
    StructField("create_date", StringType(), True), \
    StructField("last_updated_time", TimestampType(), True)
  ])

#Create dataframe from the input data and the corresponding schema
updateDF = spark.createDataFrame(data=InputData,schema=schema)

In [14]:
#Now we will Update/Insert the data to HUDI dataset on S3, instead of insert, we will use "upsert" and instead of overwrite for mode, we will use "append"

updateDF.write \
.format('org.apache.hudi') \
.option('hoodie.datasource.write.operation', 'upsert') \
.options(**hudiOptions) \
.mode('append') \
.save('s3://emr-studio-emr-on-eks/hudi-tables/')

In [15]:
# Lets check the HUDI dataset if the record was updated and the new record was inserted or not
# You should see cust_name and cust_age for cust_i=1 is updated and a new record cust_id=7 is inserted.
#also notice that the last_updated_time is also updated for cust_id=1

snapshotQueryDF = spark.read \
    .format('org.apache.hudi') \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*')
    
snapshotQueryDF.select("cust_id", "cust_name", "cust_age", "cust_loc", "create_date", "last_updated_time").orderBy("cust_id").show()

+-------+--------------+--------+--------+-----------+-------------------+
|cust_id|     cust_name|cust_age|cust_loc|create_date|  last_updated_time|
+-------+--------------+--------+--------+-----------+-------------------+
|      1|Prasad S Nadig|      30|      NJ| 2022-01-01|2022-11-28 15:07:58|
|      2|      Ethereum|      80|      NY| 2022-01-02|2022-11-28 15:06:58|
|      3|        Cosmos|      25|      PA| 2022-01-03|2022-11-28 15:06:58|
|      4|        Solana|      55|      MD| 2022-01-04|2022-11-28 15:06:58|
|      5|       Carnado|      15|      TX| 2022-01-05|2022-11-28 15:06:58|
|      6|          Link|      45|      NJ| 2022-01-06|2022-11-28 15:06:58|
|      7|      Compound|      20|      NJ| 2022-01-07|2022-11-28 15:07:58|
+-------+--------------+--------+--------+-----------+-------------------+



## DELETE

In [16]:
#HUDI alllows you to delete records just like traditional RDBMS, so let's delete a record
deleteDF = snapshotQueryDF.where("cust_id==6")

In [17]:
#Write to HUDI dataset to apply the deletes
deleteDF.write \
.format('org.apache.hudi') \
.option("hoodie.datasource.write.payload.class", "org.apache.hudi.common.model.EmptyHoodieRecordPayload") \
.options(**hudiOptions) \
.mode('append') \
.save('s3://emr-studio-emr-on-eks/hudi-tables/')

In [18]:
#Notice that cust_id=6 has been permanently deleted from the dataset
deleteReadDF = spark.read \
    .format('org.apache.hudi') \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*')
    
deleteReadDF.select("cust_id", "cust_name", "cust_age", "cust_loc", "create_date", "last_updated_time").orderBy("cust_id").show()


+-------+--------------+--------+--------+-----------+-------------------+
|cust_id|     cust_name|cust_age|cust_loc|create_date|  last_updated_time|
+-------+--------------+--------+--------+-----------+-------------------+
|      1|Prasad S Nadig|      30|      NJ| 2022-01-01|2022-11-28 15:07:58|
|      2|      Ethereum|      80|      NY| 2022-01-02|2022-11-28 15:06:58|
|      3|        Cosmos|      25|      PA| 2022-01-03|2022-11-28 15:06:58|
|      4|        Solana|      55|      MD| 2022-01-04|2022-11-28 15:06:58|
|      5|       Carnado|      15|      TX| 2022-01-05|2022-11-28 15:06:58|
|      7|      Compound|      20|      NJ| 2022-01-07|2022-11-28 15:07:58|
+-------+--------------+--------+--------+-----------+-------------------+



## Time Travel - Point in time Query

In [3]:
spark.read  \
    .format("hudi") \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*') \
    .createOrReplaceTempView("hudi_snapshot")

### Commit time

In [6]:
# Hudi stores commit time for each DML operation performed on the "recordKey"in its metadata files. 
# Below query will fetch commit time for INSERT and for UPSERT by recordKey
spark.sql("select _hoodie_commit_time as commitTime, _hoodie_record_key as primaryKey, _hoodie_partition_path as partition from  hudi_snapshot order by commitTime").show()

+-----------------+----------+----------+
|       commitTime|primaryKey| partition|
+-----------------+----------+----------+
|20221128150707999|         2|2022-01-02|
|20221128150707999|         3|2022-01-03|
|20221128150707999|         4|2022-01-04|
|20221128150707999|         5|2022-01-05|
|20221128150801565|         1|2022-01-01|
|20221128150801565|         7|2022-01-07|
|20221128175805365|         8|2022-01-01|
+-----------------+----------+----------+



In [8]:
#Lets get distinct commit time from the Hudi dataset
commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_snapshot order by commitTime").limit(50).collect()))

In [10]:
#print to verify the distinct values for commits
print(commits)

['20221128150707999', '20221128150801565', '20221128175805365']


In [9]:
#Set parameters for Hudi options
startTime = "000" # Fetches all available commits since start.
endTime = commits[len(commits) - 2] # Fetches the initial commit time

# Define Hudi options to query point in time data with a start and end time
time_travel_options = {
    'hoodie.datasource.query.type': 'incremental',
    'hoodie.datasource.read.end.instanttime': endTime,
    'hoodie.datasource.read.begin.instanttime': startTime
}

# get the initial table before upsert and delete (Original Inserts)
df_time_travel_read = spark.read.format("hudi") \
    .options(**time_travel_options)  \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*') \
    .show()

+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+--------+--------+-----------+-------------------+--------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|cust_id|     cust_name|cust_age|cust_loc|create_date|  last_updated_time|phone_no|
+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+--------+--------+-----------+-------------------+--------+
|  20221128150801565|20221128150801565...|                 1|            2022-01-01|a99af538-8324-4c1...|      1|Prasad S Nadig|      30|      NJ| 2022-01-01|2022-11-28 15:07:58|    null|
|  20221128150801565|20221128150801565...|                 7|            2022-01-07|65af084c-7eff-40e...|      7|      Compound|      20|      NJ| 2022-01-07|2022-11-28 15:07:58|    null|
|  20221128150707999|20221128150707999...|                 2

### Incremental Query

In [41]:
startTime = commits[len(commits) - 2] # fetch commit time for incremental data (UPSERT)

# fetch incremental data after initial insert, startTime represents the commit time for UPSERT
incremental_read_options = {
    'hoodie.datasource.query.type': 'incremental',
    'hoodie.datasource.read.begin.instanttime': startTime
}

df_customer_incremental_read = spark.read.format("hudi") \
    .options(**incremental_read_options)  \
    .load('s3://emr-studio-emr-on-eks/hudi-tables' + '/*/*') \
    .show()

+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+--------+--------+-----------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|cust_id|     cust_name|cust_age|cust_loc|create_date|  last_updated_time|
+-------------------+--------------------+------------------+----------------------+--------------------+-------+--------------+--------+--------+-----------+-------------------+
|  20221128150801565|20221128150801565...|                 1|            2022-01-01|a99af538-8324-4c1...|      1|Prasad S Nadig|      30|      NJ| 2022-01-01|2022-11-28 15:07:58|
|  20221128150801565|20221128150801565...|                 7|            2022-01-07|65af084c-7eff-40e...|      7|      Compound|      20|      NJ| 2022-01-07|2022-11-28 15:07:58|
+-------------------+--------------------+------------------+----------------------+--------------------+